# Universe Simulation 1: Newtonian Particles

## Code

In [ ]:
import numpy as np

In [ ]:
from collections import deque

In [ ]:
class NewtonianParticleSim:
    def __init__(self, dt=0.01, m=[1], r=None, x0=[[0,0,0]], v0=[[0,0,0]], F=lambda t,x,v: 0, G=0, q=[0], ke=0, boundary=None):
        self.n =  np.array(x0).shape[0]
        self.dim = np.array(x0).shape[-1]
        self.dt = dt
        self.m = np.array(m, dtype='double')
        self.r = np.array(r, dtype='double') if r is not None else r
        self.x0 = np.array(x0, dtype='double')
        self.v0 = np.array(v0, dtype='double')
        self.F = F
        self.G = G
        self.q = np.array(q, dtype='double')
        self.ke = ke
        self.boundary = boundary
        
        self.reset()
    
    def reset(self):
        self.t = 0
        self.x = self.x0.copy()
        self.v = self.v0.copy()
    
    def step(self):
        if self.boundary is not None:
            for idim in range(self.dim):
                mask = self.x[:,idim] < self.boundary[2*idim]
                self.v[mask,idim] = np.abs(self.v[mask,idim])
                mask = self.x[:,idim] > self.boundary[2*idim+1]
                self.v[mask,idim] = -np.abs(self.v[mask,idim])
        
        if self.r is not None:
            r_psum = self.r[:,np.newaxis] + self.r[:,np.newaxis]
            m_ratio = 2 * self.m[np.newaxis,:] / (self.m[np.newaxis,:] + self.m[:,np.newaxis]) 
            x_pdiff = self.x[np.newaxis,:] - self.x[:,np.newaxis]
            v_pdiff = self.v[np.newaxis,:] - self.v[:,np.newaxis]
            coeff = m_ratio * np.sum(x_pdiff * v_pdiff, axis=-1) / (np.sum(x_pdiff * x_pdiff, axis=-1) + np.eye(self.n,self.n))
            coeff *= coeff < 0 # don't collide into
            coeff *= np.linalg.norm(x_pdiff, axis=-1) <= r_psum # collision by distance
            dv = np.sum(coeff[:,:,np.newaxis] * x_pdiff, 1)
            self.v += dv

        Fg = np.zeros((self.n,self.dim), dtype='double')
        if self.G != 0:
            m_pprod = self.m[np.newaxis,:] * self.m[:,np.newaxis]
            x_pdiff = self.x[np.newaxis,:] - self.x[:,np.newaxis]
            coeff = self.G * m_pprod / (np.linalg.norm(x_pdiff, axis=-1)**3 + np.eye(self.n,self.n))
            Fg = np.sum(coeff[:,:,np.newaxis] * x_pdiff, 1)

        Fe = np.zeros((self.n,self.dim), dtype='double')
        if self.ke != 0:
            q_pprod = self.q[np.newaxis,:] * self.q[:,np.newaxis]
            x_pdiff = self.x[np.newaxis,:] - self.x[:,np.newaxis]
            coeff = -self.ke * q_pprod / (np.linalg.norm(x_pdiff, axis=-1)**3 + np.eye(self.n,self.n))
            Fe = np.sum(coeff[:,:,np.newaxis] * x_pdiff, 1)

        self.v += (self.F(self.t,self.x,self.v) + Fg + Fe) / self.m[:,np.newaxis] * self.dt
        self.x += self.v * self.dt
        self.t += self.dt
    
    def to(self, tf):
        while self.t < tf:
            self.step()
    
    def energy(self):
        kinetic = np.sum(self.m[:,np.newaxis] * self.v * self.v) / 2
        potential = 0
        
        if self.G != 0:
            m_pprod = self.m[np.newaxis,:] * self.m[:,np.newaxis]
            x_pdiff = self.x[np.newaxis,:] - self.x[:,np.newaxis]
            potentials = self.G * m_pprod / (np.linalg.norm(x_pdiff, axis=-1) + np.eye(self.n,self.n))
            potentials *= ~np.eye(self.n, self.n, dtype='bool')
            potential -= potentials.sum()/2
        
        return kinetic + potential

### Visualizing

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.collections import PatchCollection

In [ ]:
from IPython.display import HTML

## Kinematics
https://en.wikipedia.org/wiki/Kinematics
$$ \frac{\text{d}\vec{x}}{\text{d}t} = \vec{v} $$
$$ \frac{\text{d}\vec{v}}{\text{d}t} = \vec{a} $$



In [ ]:
kinematics_sim = NewtonianParticleSim(dt=0.01, m=[1], x0=[[0,0]], v0=[[1,0]])

In [ ]:
kinematics_sim.reset()
tf = 1
fps = 100

fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_aspect('equal')

scat = ax.scatter(kinematics_sim.x0[:,0],kinematics_sim.x0[:,1])

def update(frame):
    kinematics_sim.to(frame/float(fps))
    scat.set_offsets(kinematics_sim.x[:,:2])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=tf*fps, interval=1000//fps)
HTML(ani.to_jshtml())

## Dynamics
$$ \vec{F} = m\vec{a} $$

### Centripedal Motion
$$ \vec{F}_c = m \vec{a}_c = - m \frac{v^2}{r} \hat{r}  = - m \omega^2 \vec{r} $$
$$ v = \omega r $$
$$ \omega = \frac{2\pi}{T} $$

In [ ]:
r = 1
T = 1
omega = 2 * np.pi / T
print("Angular velocity:", omega)
v = omega * r
print("Tangential velocity:", v)

centripedal_force_sim = NewtonianParticleSim(dt=0.01, m=[1], x0=[[r,0]], v0=[[0,v]], F=lambda t,x,v: -omega**2*x)

In [ ]:
centripedal_force_sim.reset()
fps = 100

fig, ax = plt.subplots()
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')

scat = ax.scatter(centripedal_force_sim.x0[:,0], centripedal_force_sim.x0[:,1])

def update(frame):
    centripedal_force_sim.to(frame/float(fps))
    scat.set_offsets(centripedal_force_sim.x[:,:2])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(T*fps), interval=1000//fps)
HTML(ani.to_jshtml())

## Boundary Collisions
- https://en.wikipedia.org/wiki/Coefficient_of_restitution
- https://en.wikipedia.org/wiki/Elastic_collision

In [ ]:
g = np.array([0,-8])
boundary_collision_sim = NewtonianParticleSim(
    dt=0.01, 
    m=[1], 
    x0=[[0,0]], 
    v0=[[1,0]], 
    F=lambda t,x,v: g, 
    boundary=[-1,1]*2
)

In [ ]:
boundary_collision_sim.reset()
fps = 100
tail_frames = 100

fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_aspect('equal')

scat = ax.scatter(boundary_collision_sim.x0[:,0], boundary_collision_sim.x0[:,1])

tailx,taily = deque([boundary_collision_sim.x[0,0]]),deque([boundary_collision_sim.x[0,1]])
tail, = ax.plot(tailx,taily, zorder=-1, c='lightgrey')

def update(frame):
    boundary_collision_sim.to(frame/float(fps))
    scat.set_offsets(boundary_collision_sim.x[:,:2])
    tailx.append(boundary_collision_sim.x[0,0])
    taily.append(boundary_collision_sim.x[0,1])
    while len(tailx) > tail_frames:
        tailx.popleft()
        taily.popleft()
    tail.set_data(tailx,taily)

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(4*fps), interval=1000//fps)
HTML(ani.to_jshtml())

## Inter-Particle Collisions

$$ \Delta_{j} \vec{v}_{i} = 
- \frac{m_{j}}{m_{i} + m_{j}} 
\frac{\left \langle \vec{v}_{j} - \vec{v}_{i}, \vec{x}_{j} - \vec{x}_{i} \right \rangle}
{\left|\vec{x}_{j} - \vec{x}_i\right|^{2}} \left(\vec{x}_{j} - \vec{x}_i\right) $$

In [ ]:
collision_sim = NewtonianParticleSim(dt=0.01, m=[1,2], x0=[[-1,0],[1,0]], v0=[[1,0],[-1,0]], r=[0.25,0.25])

In [ ]:
collision_sim.reset()
fps = 100

fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_aspect('equal')

pts = PatchCollection([plt.Circle(x, radius=r) for x,r in zip(collision_sim.x[:,:2], collision_sim.r)])
ax.add_collection(pts)

def update(frame):
    collision_sim.to(frame/float(fps))
    pts.set_paths([plt.Circle(x, radius=r) for x,r in zip(collision_sim.x[:,:2], collision_sim.r)])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(2*fps), interval=1000//fps)
HTML(ani.to_jshtml())

### Kinetic theory of gases (Ideal Gas)
https://en.wikipedia.org/wiki/Kinetic_theory_of_gases

In [ ]:
n = 20
gas_sim = NewtonianParticleSim(
    dt=0.01, 
    m=np.ones(n), 
    x0=2*np.random.rand(n,2)-1, 
    v0=2*np.random.rand(n,2)-1, 
    r=np.ones(n)*0.05,
    boundary=[-1,1]*2#,
    #F=lambda t,x,v: np.array([[0,-4]],dtype='double')
)

In [ ]:
gas_sim.reset()
fps = 100

fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_aspect('equal')

pts = PatchCollection([plt.Circle(x, radius=r) for x,r in zip(gas_sim.x[:,:2], gas_sim.r)])
ax.add_collection(pts)

def update(frame):
    gas_sim.to(frame/float(fps))
    pts.set_paths([plt.Circle(x, radius=r) for x,r in zip(gas_sim.x[:,:2], gas_sim.r)])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(5*fps), interval=1000//fps)
HTML(ani.to_jshtml())

## Newtonian Gravity
- https://en.wikipedia.org/wiki/Newton%27s_law_of_universal_gravitation

$$ \vec{F}_{ij} = -\frac{G m_{i} m_{j}}{\left|\vec{x}_{j} - \vec{x}_i\right|^{3}} \left(\vec{x}_{j} - \vec{x}_i\right) $$

### Earth-Moon

In [ ]:
s_day = 60 * 60 * 24
earth_moon_sim = NewtonianParticleSim(
    dt = 0.01, # time units are days
    m = [
        5.972168*10**24, # Earth mass (kg)
         7.346*10**22  # Moon mass (kg)
    ],
    x0 = [
        [0,0,0],
        [384399,0,0] # Moon semi-major axis (km)
    ], 
    v0 = [
        [0,0,0],
        [0,1.022 * s_day,0] # Moon avg orbital speed (km/s) -> (km/day)
    ], 
    G = 6.67430*10**-11 * 1000**-3 * s_day**2 # gravitation constant (m**3/kg/s**2) -> (km**3/kg/day**2)
)

In [ ]:
earth_moon_sim.reset()
fps = 100
period = 30

fig, ax = plt.subplots()
ax.set_xlim(-1000*1000, 1000*1000)
ax.set_ylim(-1000*1000, 1000*1000)
ax.set_aspect('equal')

scat = ax.scatter(earth_moon_sim.x0[:,0], earth_moon_sim.x0[:,1])

def update(frame):
    earth_moon_sim.to(period*frame/float(fps))
    scat.set_offsets(earth_moon_sim.x[:,:2])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(1*fps), interval=1000/fps)
HTML(ani.to_jshtml())

### Inner Solar System
- https://en.wikipedia.org/wiki/Solar_System
- https://en.wikipedia.org/wiki/Sun
- https://en.wikipedia.org/wiki/Mercury_(planet)
- https://en.wikipedia.org/wiki/Venus
- https://en.wikipedia.org/wiki/Earth


In [ ]:
s_day = 60 * 60 * 24
solar_system_sim = NewtonianParticleSim(
    dt = 1, # time units are days
    m = [
        1.9885*10**30, # Sun mass (kg)
        3.3011*10**23, # Mercury mass (kg)
        4.8675*10**24, # Venus mass (kg)
        5.972168*10**24, # Earth mass (kg)
        6.4171*10**23, # Mars mass (kg)
    ],
    x0 = [
        [0,0,0], # heliocentric
        [57.91*1000*1000,0,0], # Mercury semi-major axis (km)
        [108.21*1000*1000,0,0], # Venus semi-major axis (km)
        [149598023,0,0], # Earth semi-major axis (km)
        [227939366,0,0], # Mars semi-major axis (km)
    ], 
    v0 = [
        [0,0,0], # heliocentric
        [0,47.36*s_day,0], # Mercury avg orbital speed (km/s) -> (km/day)
        [0,35.02*s_day,0], # Venus avg orbital speed (km/s) -> (km/day)
        [0,29.7827*s_day,0], # Earth avg orbital speed (km/s) -> (km/day)
        [0,24.07*s_day,0], # Mars avg orbital speed (km/s) -> (km/day)
    ], 
    G = 6.67430*10**-11 * 1000**-3 * s_day**2 # gravitation constant (m**3/kg/s**2) -> (km**3/kg/day**2)
)

In [ ]:
solar_system_sim.reset()
fps = 100
res = 100

fig, ax = plt.subplots()
ax.set_xlim(-300*1000*1000, 300*1000*1000)
ax.set_ylim(-300*1000*1000, 300*1000*1000)
ax.set_aspect('equal')

for r in [57.91*1000*1000, 108.21*1000*1000, 149598023, 227939366]:
    t = np.linspace(0,2*np.pi, res)
    x = r * np.cos(t)
    y = r * np.sin(t)
    ax.plot(x,y, c='lightgrey', zorder=-1)

scat = ax.scatter(solar_system_sim.x0[:,0], solar_system_sim.x0[:,1])

def update(frame):
    solar_system_sim.to(365*frame/float(fps))
    scat.set_offsets(solar_system_sim.x[:,:2])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(2*fps), interval=1000//fps)
HTML(ani.to_jshtml())

### Binary Star
- https://en.wikipedia.org/wiki/Eccentricity_(mathematics)
- https://en.wikipedia.org/wiki/Apsis
- https://en.wikipedia.org/wiki/Orbital_period

Total energy of the system is constant
$$
E = - \frac{G m_{0} m_{1}}{\left| \vec{x}_{1} - \vec{x}_{0} \right|} 
+ \frac{m_{0}}{2} \left| \vec{v}_{0} \right|^{2}
+ \frac{m_{1}}{2} \left| \vec{v}_{1} \right|^{2}
$$

Sub in the equal mass
$$
E = - \frac{G m^{2}}{\left| \vec{x}_{1} - \vec{x}_{0} \right|} 
+ m \left| \vec{v} \right|^{2}
$$

Equate the apoapsis and periapsis
$$
E = - \frac{G m^{2}}{2 r_{a}} + m v_{a}^{2}
= - \frac{G m^{2}}{2 r_{p}} + m v_{p}^{2}
$$

Simplify
$$
- \frac{G m}{2 r_{a}} + v_{a}^{2}
= - \frac{G m}{2 r_{p}} + v_{p}^{2}
$$

$$
\frac{G m}{2} \left( \frac{1}{r_{p}} - \frac{1}{r_{a}} \right)
= v_{p}^{2} - v_{a}^{2}
$$

The total angular moment is constant
$$
\vec{L} = m \vec{r} \times \vec{v}
$$

Equate the apoapsis and periapsis
$$
\left| \vec{L} \right| = 2 m r_{a} v_{a} = 2 m r_{p} v_{p}
$$

$$
v_{p} = \frac{r_{a}}{r_{p}} v_{a}
$$

Sub in
$$
\frac{G m}{2} \left( \frac{1}{r_{p}} - \frac{1}{r_{a}} \right)
= \left( \frac{r_{a}}{r_{p}} v_{a} \right)^{2} - v_{a}^{2}
$$

Simplify
$$
\frac{G m}{2} \left( \frac{1}{r_{p}} - \frac{1}{r_{a}} \right)
= \left( \left( \frac{r_{a}}{r_{p}} \right)^{2} - 1 \right) v_{a}^{2}
$$

$$
\frac{G m}{2} \left( r_{a} r_{p} - r_{p}^{2} \right)
= \left( r_{a}^{3} - r_{a} r_{p}^{2} \right) v_{a}^{2}
$$

$$
\frac{G m}{2} \frac{r_{a} r_{p} - r_{p}^{2}}{r_{a}^{3} - r_{a} r_{p}^{2}}
= v_{a}^{2}
$$

$$
\frac{G m}{2} \frac{r_{p}}{r_{a}} \frac{r_{a} - r_{p}}{r_{a}^{2} - r_{p}^{2}}
= v_{a}^{2}
$$

$$
\frac{G m}{2} \frac{r_{p}}{r_{a}} \frac{1}{r_{a} + r_{p}}
= v_{a}^{2}
$$

$$
\frac{G m}{4} \frac{r_{p}}{r_{a}} \frac{1}{a}
= v_{a}^{2}
$$

Orbital period
$$
T = 2 \pi {\sqrt{\frac {a^{3}}{G \left(m_{0}+m_{1}\right)}}}
$$

Sub in
$$
T = 2 \pi {\sqrt{\frac {a^{3}}{2 G m}}}
$$

In [ ]:
a = 2
b = 1.5
G = 1
m = 1

e = np.sqrt(1-(b/a)**2)
print("Eccentricity", e)
ra = (1 + e) * a
print("Apopasis", ra)
rp = (1 - e) * a
print("Periapasis", rp)
va = np.sqrt(G*m/4 * rp/ra/a)
print("Apopasis velocity", va)

T = 2 * np.pi * np.sqrt(2**3 * a**3 / (2 * G * m))
T

In [ ]:
binary_star_sim = NewtonianParticleSim(
    dt = 0.01, 
    m = [m,m], 
    x0 = [[-ra,0,0],[ra,0,0]], 
    v0 = [[0,va,0],[0,-va,0]], 
    G = G
)

In [ ]:
binary_star_sim.reset()
fps = 100
res = 100

fig, ax = plt.subplots()
ax.set_xlim(-1.2*ra, 1.2*ra)
ax.set_ylim(-1.2*ra, 1.2*ra)
ax.set_aspect('equal')

t = np.linspace(0,2*np.pi, res)
x = a * np.cos(t) - (a*e)
y = b * np.sin(t)
ax.plot(x,y, c='lightgrey', zorder=-1)

t = np.linspace(0,2*np.pi, res)
x = a * np.cos(t) + (a*e)
y = b * np.sin(t)
ax.plot(x,y, c='lightgrey', zorder=-1)

ax.scatter([0], [0], c='lightgrey')


scat = ax.scatter(binary_star_sim.x0[:,0], binary_star_sim.x0[:,1])

def update(frame):
    binary_star_sim.to(T*frame/float(fps))
    scat.set_offsets(binary_star_sim.x[:,:2])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(1*fps), interval=1000//fps)
HTML(ani.to_jshtml())

## Coulomb Force
- https://en.wikipedia.org/wiki/Coulomb%27s_law

$$ \vec{F}_{ij} = \frac{k_{e} q_{i} q_{j}}{\left|\vec{x}_{j} - \vec{x}_i\right|^{3}} \left(\vec{x}_{j} - \vec{x}_i\right) $$

### Bohr model (Classical Hydrogen Atom)
- https://en.wikipedia.org/wiki/Bohr_radius
- https://en.wikipedia.org/wiki/Bohr_model

Coulomb force is centripedal force
$$ \left| \vec{F} \right| = \frac{k_{e} q^{2}}{r^{2}} = m r \omega^{2} $$

Sub in the values
$$ \left| \vec{F} \right| = \frac{k_{e} Z e^{2}}{r^{2}} = m_{e} r \omega^{2} $$

Simplify
$$ k_{e} Z e^{2} = m_{e} r^{3} \omega^{2} $$

Angular momentum is discrete
$$ \left| \vec{L} \right| = m r^{2} \omega $$
$$ \left| \vec{L} \right| = m_{e} r^{2} \omega = n \hbar$$

Equate
$$ k_{e} Z e^{2} m_{e} r = \left( m_{e} r^{2} \omega \right)^{2} = n^{2} \hbar^{2} $$

Solve for radius and define Bohr radius
$$ r = \frac{n^{2}}{Z} \frac{\hbar^{2}}{k_{e} e^{2} m_{e}} $$
$$ r = \frac{n^{2}}{Z} a_{0} $$
$$ a_{0} = \frac{\hbar^{2}}{k_{e} e^{2} m_{e}} $$

Find electron velocity
$$ \left| \vec{L} \right| = m_{e} r v = \frac{n^{2}}{Z} m_{e} a_{0} v = n \hbar $$
$$ v = \frac{Z}{n} \frac{\hbar}{m_{e} a_{0}} $$

Find orbital period
$$ v = \omega r $$
$$ \omega = \frac{2 \pi}{T} $$

$$ T = \frac{2 \pi}{\omega} = \frac{2 \pi r}{v}$$

In [ ]:
hbar = 1.054571817*10**-34 # reduced Planck constant (kg*m**2/s)
ke = 8.9875517862*10**9 # Coulomb's constant (kg*m**3/C**2/s**2)
e = 1.602176634*10**-19 # elementary charge (C)
me = 9.1093837139*10**-31 # electron mass (kg)

a0 = hbar**2 / (ke * e**2 * me) # The Bohr radius (m)
a0

In [ ]:
Z = 1
n = 1

r = n**2 / Z * a0
v = Z / n * hbar / me / a0
print(v)
T = 2 * np.pi * r / v
print(T)
mp = 1.67262192595*10**-27 # proton mass (kg)

Bohr_sim = NewtonianParticleSim(
    dt = 0.01*T, # time units are s
    m = [
        mp, # proton mass (kg)
        me # electron mass (kg)
    ],
    q = [
        e, # proton charge (C)
        -e # electron mass (C)
    ],
    x0 = [
        [0,0,0],
        [r,0,0] # Atom radius (m)
    ], 
    v0 = [
        [0,0,0],
        [0,v,0] # electron orbital speed (m/s)
    ], 
    ke = ke # Coulomb's constant (kg*m**3/C**2/s**2)
)

In [ ]:
Bohr_sim.reset()
fps = 100

fig, ax = plt.subplots()
ax.set_xlim(-10*a0, 10*a0)
ax.set_ylim(-10*a0, 10*a0)
ax.set_aspect('equal')

scat = ax.scatter(Bohr_sim.x0[:,0], Bohr_sim.x0[:,1])

for n in range(1,3+1):
    t = np.linspace(0,2*np.pi, res)
    rn = n**2 / Z * a0
    x = rn * np.cos(t)
    y = rn * np.sin(t)
    ax.plot(x,y, c='lightgrey', zorder=-1)

def update(frame):
    Bohr_sim.to(T*frame/float(fps))
    scat.set_offsets(Bohr_sim.x[:,:2])

plt.close(fig)
ani = FuncAnimation(fig, update, frames=int(1*fps), interval=1000/fps)
HTML(ani.to_jshtml())

## ODEs and the IVP
$$ \frac{\text{d}\vec{x}}{\text{d}t} = f\left(t,\vec{x}\right) $$

$$ \frac{\text{d}\vec{x}}{\text{d}t} = \vec{v} $$
$$ \frac{\text{d}\vec{v}}{\text{d}t} = \frac{\vec{F}}{m} $$